# IPyGPULogger Demo

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

In [3]:
def consume_cpu_ram(n): return np.ones((n, n))
def consume_gpu_ram(n): return torch.ones((n, n)).cuda()
def consume_cpu_ram_128mb(): return consume_cpu_ram(2**12)
def consume_gpu_ram_256mb(): return consume_gpu_ram(2**13)

In [4]:
from ipygpulogger import IPyGPULogger
il = IPyGPULogger().start()

RAM: Consumed Peaked  Used Total | Exec time 0.001s
CPU:        0      0     2150 MB |
GPU:        0      0     2156 MB |


In [ ]:
# consume, consume more
a1 = consume_cpu_ram_128mb()
a2 = consume_cpu_ram_128mb()

RAM: Consumed Peaked  Used Total | Exec time 0.070s
CPU:      256      0     2406 MB |
GPU:        0      0     2156 MB |


In [ ]:
# consume, consume more, then release some (temporarily requires double the memory)
a1 = consume_cpu_ram_128mb()
a2 = consume_cpu_ram_128mb()
del a1

RAM: Consumed Peaked  Used Total | Exec time 0.081s
CPU:      128    128     2278 MB |
GPU:        0      0     2156 MB |


In [ ]:
# consume, release, then consume more (half the memory is needed at the peak)
a1 = consume_cpu_ram_128mb()
del a1
a2 = consume_cpu_ram_128mb()

RAM: Consumed Peaked  Used Total | Exec time 0.078s
CPU:      128      0     2278 MB |
GPU:        0      0     2156 MB |


In [ ]:
# consume, consume more, release all (all memory get reclaimed)
b1 = consume_cpu_ram_128mb()
b2 = consume_cpu_ram_128mb()
del b1
del b2

RAM: Consumed Peaked  Used Total | Exec time 0.076s
CPU:        0    256     2278 MB |
GPU:        0      0     2156 MB |


In [ ]:
# same on gpu
c1 = consume_gpu_ram_256mb()

RAM: Consumed Peaked  Used Total | Exec time 0.093s
CPU:        0      0     2278 MB |
GPU:      256      0     2412 MB |


In [ ]:
# direct data accessor (in not rounded MBs)
cpu_mem, gpu_mem, time_data = il.data
cpu_mem
gpu_mem
time_data
gpu_mem.used_delta

IPyGPULoggerMemory(used_delta=0.005390167236328125, peaked_delta=0.0626678466796875, used_total=2278)

IPyGPULoggerMemory(used_delta=256, peaked_delta=0, used_total=2412)

IPyGPULoggerTime(time_delta=0.09250283241271973)

256

RAM: Consumed Peaked  Used Total | Exec time 0.008s
CPU:        0      0     2278 MB |
GPU:        0      0     2412 MB |


In [ ]:
il.stop()

In [ ]:
# there should be no log appearing, after the logger has been stopped
a = consume_cpu_ram_128mb()

In [ ]:
%%javascript # prevent committing an unsaved notebook
_=IPython.notebook.save_notebook()